# Explore source measurement on a custom coadd

This NB uses the same procedure as in notebook 05_Introduction_to_Source_Detection.
We've already made a custom coadd with just six input visits, and are using that here.

> **Help Question:** NB 05 uses a calexp, not a coadd. Still OK to base the code below on NB 05?

> **Help Question:** Can we skip 2, CharacterizeImageTask, because our coadd is characterized?

Now want to run source detection, deblending, and measurement on the coadd.

> **Help Question:** In 4, the debelending returns an assertion error about the schema not matching even when we make sure they match at the start. Unsure how to fix. Same seems to happen in 5.

> **Help Question:** Maybe there's a better way to do the coadd source detection, deblending, and measurement?

In the DRP.yaml file there are the following, maybe we just add them...:
 - detection: lsst.pipe.tasks.multiBand.DetectCoaddSourcesTask
 - deblend: lsst.pipe.tasks.deblendCoaddSourcesPipeline.DeblendCoaddSourcesMultiTask
 - measure: lsst.pipe.tasks.multiBand.MeasureMergedCoaddSourcesTask

In [1]:
import time
import numpy as np
import pandas

import lsst.geom
import lsst.afw.display as afwDisplay

from lsst.daf.butler import Butler

import lsst.afw.table as afwTable
from lsst.meas.algorithms.detection import SourceDetectionTask

from lsst.meas.deblender import SourceDeblendTask

from lsst.meas.base import SingleFrameMeasurementTask
import lsst.daf.base as dafBase

## 1. Get the coadd I made in draft_Create_Custom_Coadd.ipynb

In [2]:
config = 'dp02'
collection = 'u/melissagraham/coadd_recreation_nb/20220715T220907Z'

butler = Butler(config, collections=collection)

my_dataId = {'band': 'i', 'tract': 4431, 'patch': 17}
my_coadd = butler.get('deepCoadd', dataId=my_dataId)

See, it's a special coadd.

In [3]:
my_coadd_inputs = butler.get("deepCoadd.coaddInputs", my_dataId)
my_coadd_inputs.visits.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,goodpix,weight,filter
,pix,pix,pix,pix,,,
int64,int32,int32,int32,int32,int32,float64,str32
919515,11900,7900,16099,12099,8982709,3.4656688819793495,i_sim_1.4
924057,11900,7900,16099,12099,16098179,4.384267091685517,i_sim_1.4
924085,11900,7900,16099,12099,831332,4.446833161599578,i_sim_1.4
924086,11900,7900,16099,12099,16136708,4.550420295334223,i_sim_1.4
929477,11900,7900,16099,12099,16280498,4.051326013718346,i_sim_1.4
930353,11900,7900,16099,12099,16076133,3.7685753871220466,i_sim_1.4


## 2. Characterize Image Task

> **Help Question:** Is it necessary to run CharacterizeImageTask on the coadd? It seems the image already has psf, is already characterized?

In [4]:
my_coadd_bbox = butler.get("deepCoadd.bbox", dataId=my_dataId)
x_val = my_coadd_bbox.beginX + 200
y_val = my_coadd_bbox.beginY + 200
point = lsst.geom.Point2D(x_val, y_val)

psf = my_coadd.getPsf()
psfShape = psf.computeShape(point)
sigma = psfShape.getDeterminantRadius()
pixelScale = my_coadd.getWcs().getPixelScale().asArcseconds()
print('psf fwhm = {:.2f} arcsec at x = {:.0f}, y = {:.0f}'.format(sigma*pixelScale*2.355,x_val,y_val))

psf fwhm = 0.84 arcsec at x = 12100, y = 8100


If it's necessary, do like this?

In [5]:
# from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask

# ci_config = CharacterizeImageTask.ConfigClass()
# ci_config.psfIterations = 1
# charImageTask = CharacterizeImageTask(config=ci_config)

# result = charImageTask.run(my_coadd)

## 3. Configure Source Detection, Deblend, and Measurement Tasks

In [6]:
schema = afwTable.SourceTable.makeMinimalSchema()
tab = afwTable.SourceTable.make(schema)

In [7]:
schema

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
)

In [8]:
sd_config = SourceDetectionTask.ConfigClass()
sd_config.thresholdValue = 5
sd_config.thresholdType = "stdev"

sourceDetectionTask = SourceDetectionTask(schema=schema, config=sd_config)

In [9]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

In [10]:
algMetadata = dafBase.PropertyList()
print('algMetadata: ')
algMetadata

algMetadata: 


In [11]:
sm_config = SingleFrameMeasurementTask.ConfigClass()

sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema,
                                                   config=sm_config,
                                                   algMetadata=algMetadata)

## 4. Run Source Detection Task

In [12]:
%%time
result = sourceDetectionTask.run(tab, my_coadd)

CPU times: user 4.65 s, sys: 943 ms, total: 5.59 s
Wall time: 5.59 s


In [13]:
sources = result.sources

In [14]:
print(len(sources), result.numPosPeaks)

3871 4878


In [15]:
# sources.asAstropy()

## 5. Run Source Deblending

https://pipelines.lsst.io/modules/lsst.meas.deblender/index.html

https://pipelines.lsst.io/modules/lsst.meas.deblender/tasks/lsst.meas.deblender.SourceDeblendTask.html

> **Help Question:** `sourceDeblendTask.run` exists with an assertion error about the schemas even when we make sure the input schema matches before we start. Not sure how to fix.

SourceDeblendTask modifies the source catalog in-place.

In [16]:
assert sources.getSchema() == schema

AssertionError: 

Attempt to fix that by redefining schema and sourceDeblendTask.

In [17]:
schema = sources.getSchema()

In [18]:
assert sources.getSchema() == schema

In [19]:
sourceDeblendTask = SourceDeblendTask(schema=schema)

Does not seem to help, below we still get the error:
> `--> 275 assert sources.getSchema() == self.schema`

In [20]:
%%time
sourceDeblendTask.run(my_coadd, sources)

AssertionError: 

In [21]:
print(len(sources))

3871


## 6. Run Source Measurement

https://pipelines.lsst.io/modules/lsst.meas.base/tasks/lsst.meas.base.sfm.SingleFrameMeasurementTask.html

In [22]:
%%time
sourceMeasurementTask.run(measCat=sources, exposure=my_coadd)

AssertionError: 